# gdsfactory downloads

You can plot the downloads for gdsfactory over the last year.

In [ ]:
import datetime

import plotly.graph_objects as go
import requests

downloads0 = 0


def get_total_downloads(package_name):
    statistics = []
    end_date = datetime.date.today()

    while True:
        url = f"https://pypistats.org/api/packages/{package_name}/overall"
        response = requests.get(url, params={"last_day": end_date})
        data = response.json()

        if response.status_code != 200:
            return None

        statistics.extend(
            [(entry["date"], entry["downloads"]) for entry in data["data"]]
        )
        if "next_day" in data:
            end_date = data["next_day"]
        else:
            break
    statistics.sort(key=lambda x: x[0])  # Sort by date
    dates, downloads = zip(*statistics)
    cumulative_downloads = [
        sum(downloads[: i + 1]) + downloads0 for i in range(len(downloads))
    ]

    return dates, cumulative_downloads


# Replace 'gdsfactory' with the package you want to check
package_name = "gdsfactory"
dates, cumulative_downloads = get_total_downloads(package_name)

if dates and cumulative_downloads:
    fig = go.Figure(data=go.Scatter(x=dates, y=cumulative_downloads))
    fig.update_layout(
        xaxis=dict(title="Date", tickformat="%Y-%m-%d", tickangle=45, showgrid=True),
        yaxis=dict(title="Total Downloads", showgrid=True),
        title=f"Total Downloads - {package_name}",
    )
    fig.update_layout(autosize=False, width=800, height=600)
    fig.show()
else:
    print(f"Failed to retrieve download statistics for package '{package_name}'.")

## dependencies

In [ ]:
import networkx as nx
import pkg_resources
import plotly.graph_objects as go


def build_dependency_graph(package_name):
    graph = nx.DiGraph()
    visited = set()

    def traverse_dependencies(package):
        if package not in visited:
            visited.add(package)
            graph.add_node(package)

            try:
                dependencies = pkg_resources.get_distribution(package).requires()
                for dependency in dependencies:
                    graph.add_edge(package, dependency)
                    traverse_dependencies(dependency)
            except pkg_resources.DistributionNotFound:
                # Package is not installed or cannot be found
                pass

    traverse_dependencies(package_name)
    return graph


# Specify the name of the package you want to build the dependency graph for
package_name = "gdsfactory"

# Build the dependency graph
dependency_graph = build_dependency_graph(package_name)

# Create nodes and edges
nodes = dependency_graph.nodes()
edges = dependency_graph.edges()

# Create Plotly nodes
node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode="markers",
    hoverinfo="text",
    marker=dict(size=15, color="lightblue", line_width=2),
)

# Create Plotly edges
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=1, color="gray"),
    hoverinfo="none",
    mode="lines",
)

# Assign positions to nodes
pos = nx.spring_layout(dependency_graph, k=0.2)
for node in nodes:
    x, y = pos[node]
    node_trace["x"] += (x,)
    node_trace["y"] += (y,)
    node_trace["text"] += (node,)

# Assign positions to edges
for edge in edges:
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace["x"] += (x0, x1, None)
    edge_trace["y"] += (y0, y1, None)

# Create the layout for the graph
layout = go.Layout(
    title=f"Dependency Graph for {package_name}",
    title_font=dict(size=20),
    showlegend=False,
    hovermode="closest",
    margin=dict(b=20, l=5, r=5, t=40),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
)

# Combine nodes and edges into a data list
data = [edge_trace, node_trace]

# Create the figure and plot the graph
fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
len(nodes)